In [22]:
# import all utilities

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [23]:
# get the html data
data = requests.get('https://en.wikipedia.org/wiki/List_of_postcode_areas_in_the_United_Kingdom').text

In [24]:
print(type(data))

<class 'str'>


In [25]:
#parsing
soup = BeautifulSoup(data, 'html.parser')

In [26]:
# create three lists to store table data
Postcode_area = []
Postcode_area_name = []
Code_formation = []

# append the data 
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        Postcode_area.append(cells[0].text.rstrip('\n'))
        Postcode_area_name.append(cells[1].text.rstrip('\n'))
        Code_formation.append(cells[2].text.rstrip('\n')) 

In [27]:
# toronto_df
uk_df = pd.DataFrame({"Postcode_area": Postcode_area,
                           "Postcode_area_name": Postcode_area_name,
                           "Code_formation": Code_formation})

uk_df.head()

,Postcode_area,Postcode_area_name,Code_formation
0,AB,Aberdeen,
1,AL,St Albans,
2,B,Birmingham,
3,BA,Bath,
4,BB,Blackburn,


In [28]:
# print the number of rows of the cleaned dataframe
uk_df.shape

(121, 3)

In [29]:
!pip install geocoder

import geocoder # to get coordinates

# define a function to get coordinates
def get_latlng(Postcode_area_name):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, UK'.format(Postcode_area_name))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(Postcode_area_name) for Postcode_area_name in uk_df["Postcode_area_name"].tolist() ]
coords

[[57.14548000000008, -2.1027199999999766],
 [51.75360000000006, -0.3372999999999706],
 [52.47891000000004, -1.9059199999999805],
 [51.38503000000003, -2.361319999999978],
 [53.748590000000036, -2.479919999999936],
 [53.795720000000074, -1.758299999999963],
 [50.720390000000066, -1.8809199999999464],
 [53.578430000000026, -2.42665999999997],
 [50.82881000000003, -0.13419999999996435],
 [51.40573000000006, 0.015060000000062246],
 [51.453790000000026, -2.59167999999994],
 [54.59534000000008, -5.934549999999945],
 [54.89647000000008, -2.9362899999999286],
 [52.20992000000007, 0.11161000000004151],
 [51.48127000000005, -3.180719999999951],
 [53.18813000000006, -2.8943499999999744],
 [51.73207000000008, 0.4779600000000528],
 [51.88531000000006, 0.9035200000000714],
 [51.372340000000065, -0.09944999999993342],
 [51.27598000000006, 1.0756000000000654],
 [52.40518000000003, -1.5085599999999317],
 [53.10085000000004, -2.446969999999965],
 [51.44462000000004, 0.2205700000000661],
 [56.46127000000

In [30]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
uk_df['Latitude'] = df_coords['Latitude']
uk_df['Longitude'] = df_coords['Longitude']

In [31]:
print(uk_df.columns)

print(uk_df.head())



Index(['Postcode_area', 'Postcode_area_name', 'Code_formation', 'Latitude',
       'Longitude'],
      dtype='object')
  Postcode_area Postcode_area_name Code_formation  Latitude  Longitude
0            AB           Aberdeen                 57.14548   -2.10272
1            AL          St Albans                 51.75360   -0.33730
2             B         Birmingham                 52.47891   -1.90592
3            BA               Bath                 51.38503   -2.36132
4            BB          Blackburn                 53.74859   -2.47992


In [32]:
Address = 'UK'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(Address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of UK are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of UK are 54.7023545, -3.2765753.


In [33]:
#### Create a map of New York with neighborhoods superimposed on top.

map_uk = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, Postcode_area_name in zip(uk_df['Latitude'], uk_df['Longitude'], uk_df['Postcode_area_name']):
    label = '{}'.format(Postcode_area_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_uk)  
    
map_uk

In [44]:
CLIENT_ID = 'PSLAWMO4O5EKEIRADUGG03FGQZ4XORKFQHYMBHF1I1TX2FAE' # your Foursquare ID
CLIENT_SECRET = 'KHMONH5AGBMC0K13A3ZZLBZFSPUOFSKZJQKTPPU5GRMXFOL5' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PSLAWMO4O5EKEIRADUGG03FGQZ4XORKFQHYMBHF1I1TX2FAE
CLIENT_SECRET:KHMONH5AGBMC0K13A3ZZLBZFSPUOFSKZJQKTPPU5GRMXFOL5


In [45]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode_area_name', 
                  'Postcode_area_name Latitude', 
                  'Postcode_area_name Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
LIMIT = 100
radius = 500

uk_venues = getNearbyVenues(names=uk_df['Postcode_area_name'],
                                   latitudes=uk_df['Latitude'],
                                   longitudes=uk_df['Longitude']
                                  )

Aberdeen
St Albans
Birmingham
Bath
Blackburn
Bradford
Bournemouth
Bolton
Brighton
Bromley
Bristol
Belfast
Carlisle
Cambridge
Cardiff
Chester
Chelmsford
Colchester
Croydon
Canterbury
Coventry
Crewe
Dartford
Dundee
Derby
Dumfries[1]
Durham
Darlington
Doncaster
Dorchester
Dudley
East London
East Central London
Edinburgh
Enfield
Exeter
Falkirk
Blackpool[1]
Glasgow
Gloucester
Guildford
Harrow
Huddersfield
Harrogate
Hemel Hempstead
Hereford
Hebrides
Hull
Halifax
Ilford
Ipswich
Inverness
Kilmarnock
Kingston upon Thames
Kirkwall
Kirkcaldy
Liverpool
Lancaster
Llandrindod Wells[1]
Leicester
Llandudno
Lincoln
Leeds
Luton
Manchester
Medway
Milton Keynes
Motherwell
North London
Newcastle upon Tyne[4][5]
Nottingham
Northampton
Newport
Norwich
North West London
Oldham
Oxford
Paisley
Peterborough
Perth
Plymouth
Portsmouth
Preston
Reading
Redhill
Romford
Sheffield
Swansea
South East London
Stevenage
Stockport
Slough
Sutton[1]
Swindon
Southampton
Salisbury
Sunderland
Southend-on-Sea
Stoke-on-Trent
South

In [48]:
# create map of uk using latitude and longitude values
map_uk = folium.Map(location=[latitude, longitude], zoom_start=5)

# add markers to map
for lat, lng, Postcode_area_name in zip(uk_df['Latitude'], uk_df['Longitude'], uk_df['Postcode_area_name']):
    label = '{}'.format(Postcode_area_name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_uk)  
    
map_uk

In [49]:
print(uk_venues.shape)
uk_venues.head()

(4843, 7)


,Postcode_area_name,Postcode_area_name Latitude,Postcode_area_name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aberdeen,57.14548,-2.10272,Belmont Filmhouse,57.147310,-2.102200,Indie Movie Theater
1,Aberdeen,57.14548,-2.10272,The Grill,57.144931,-2.104434,Whisky Bar
2,Aberdeen,57.14548,-2.10272,Cafe 52,57.146053,-2.099170,Restaurant
3,Aberdeen,57.14548,-2.10272,CUP,57.146885,-2.100738,Tea Room
4,Aberdeen,57.14548,-2.10272,Revolucion de Cuba,57.147413,-2.101209,Bar


In [50]:
uk_venues.groupby('Postcode_area_name').count().head()

,Postcode_area_name Latitude,Postcode_area_name Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode_area_name,,,,,,
Aberdeen,100,100,100,100,100,100
Bath,57,57,57,57,57,57
Belfast,100,100,100,100,100,100
Birmingham,53,53,53,53,53,53
Blackburn,20,20,20,20,20,20


In [51]:
print('There are {} uniques categories.'.format(len(uk_venues['Venue Category'].unique())))




There are 276 uniques categories.


In [140]:
# one hot encoding
uk_venues.head()
uk_onehot = pd.get_dummies(uk_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
uk_onehot['Postcode_area_name'] = uk_venues['Postcode_area_name'] 


uk_onehot.head()




,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Café,Camera Store,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Carpet Store,Casino,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Library,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Laser Tag,Latin American Restaurant,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Outdoor Sculpture,Outdoor Supply Store,Pakistani Restaurant,Park,Pastry Shop,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Piercing Parlor,Pizza Place,Planetarium,Platform,Plaza,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racecourse,Record Shop,Recording Studio,Rental Car Location,Rental Service,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Stadium,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Toy / Game Store,Trail,Train Station,Tram Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo,Postcode_area_name
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [141]:
print(uk_onehot.columns)
uk_onehot= uk_onehot.groupby(['Postcode_area_name']).sum().reset_index()

#print(uk_onehot.groupby(['Postcode_area_name']).sum())

#validation
print(uk_onehot[uk_onehot["Postcode_area_name"] == 'Aberdeen'].shape[0])
print(uk_onehot.head())



Index(['Accessories Store', 'Afghan Restaurant', 'African Restaurant',
       'American Restaurant', 'Antique Shop', 'Art Gallery', 'Art Museum',
       'Arts & Crafts Store', 'Arts & Entertainment', 'Asian Restaurant',
       ...
       'Waterfront', 'Whisky Bar', 'Wine Bar', 'Wine Shop', 'Wings Joint',
       'Women's Store', 'Yakitori Restaurant', 'Yoga Studio', 'Zoo',
       'Postcode_area_name'],
      dtype='object', length=277)
1
  Postcode_area_name  Accessories Store  Afghan Restaurant  \
0           Aberdeen                  0                  0   
1               Bath                  0                  0   
2            Belfast                  0                  0   
3         Birmingham                  0                  0   
4          Blackburn                  0                  0   

   African Restaurant  American Restaurant  Antique Shop  Art Gallery  \
0                   0                    1             0            1   
1                   0                   

In [144]:
#uk_onehot = uk_onehot[["Postcode_area_name","Shopping Mall"]]
#uk_onehot = uk_onehot[["Postcode_area_name","Art Gallery","Art Museum","Arts & Crafts Store","Arts & Entertainment"]]
uk_onehot = uk_onehot[['Postcode_area_name','Asian Restaurant']]

uk_onehot.head()

,Postcode_area_name,Asian Restaurant
0,Aberdeen,2
1,Bath,1
2,Belfast,3
3,Birmingham,0
4,Blackburn,0


In [145]:
uk_onehot.shape

(117, 2)

In [146]:
# set number of clusters
kclusters = 3

uk_grouped_clustering = uk_onehot.drop('Postcode_area_name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(uk_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 1, 0, 0, 0, 0, 2, 0, 0], dtype=int32)

In [147]:
# add clustering labels
uk_onehot.insert(0, 'Cluster Labels', kmeans.labels_)

uk_merged = uk_df.join(uk_onehot.set_index('Postcode_area_name'), on='Postcode_area_name')

uk_merged.head() # check the last columns!

,Postcode_area,Postcode_area_name,Code_formation,Latitude,Longitude,Cluster Labels,Asian Restaurant
0,AB,Aberdeen,,57.14548,-2.10272,1.0,2.0
1,AL,St Albans,,51.75360,-0.33730,2.0,1.0
2,B,Birmingham,,52.47891,-1.90592,0.0,0.0
3,BA,Bath,,51.38503,-2.36132,2.0,1.0
4,BB,Blackburn,,53.74859,-2.47992,0.0,0.0


In [195]:
# cluster visualizations
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
uk_merged = uk_merged.dropna()
uk_merged['Cluster Labels'] = uk_merged['Cluster Labels'].astype(int)

# add markers to the map
markers_colors = []
for (lat, lon, poi, cluster) in zip(uk_merged['Latitude'], uk_merged['Longitude'], uk_merged['Postcode_area_name'], uk_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [148]:
uk_merged.loc[uk_merged['Cluster Labels'] == 0, uk_merged.columns[[1] + list(range(5, uk_merged.shape[1]))]]

,Postcode_area_name,Cluster Labels,Asian Restaurant
2,Birmingham,0.0,0.0
4,Blackburn,0.0,0.0
5,Bradford,0.0,0.0
7,Bolton,0.0,0.0
8,Brighton,0.0,0.0
12,Carlisle,0.0,0.0
13,Cambridge,0.0,0.0
14,Cardiff,0.0,0.0
15,Chester,0.0,0.0
16,Chelmsford,0.0,0.0


In [149]:
uk_merged.loc[uk_merged['Cluster Labels'] == 1, uk_merged.columns[[1] + list(range(5, uk_merged.shape[1]))]]

,Postcode_area_name,Cluster Labels,Asian Restaurant
0,Aberdeen,1.0,2.0
10,Bristol,1.0,2.0
11,Belfast,1.0,3.0
18,Croydon,1.0,2.0
26,Durham,1.0,2.0
35,Exeter,1.0,2.0
40,Guildford,1.0,2.0
43,Harrogate,1.0,2.0
50,Ipswich,1.0,2.0
62,Leeds,1.0,2.0


In [150]:
uk_merged.loc[uk_merged['Cluster Labels'] == 2, uk_merged.columns[[1] + list(range(5, uk_merged.shape[1]))]]

,Postcode_area_name,Cluster Labels,Asian Restaurant
1,St Albans,2.0,1.0
3,Bath,2.0,1.0
6,Bournemouth,2.0,1.0
9,Bromley,2.0,1.0
17,Colchester,2.0,1.0
20,Coventry,2.0,1.0
29,Dorchester,2.0,1.0
32,East Central London,2.0,1.0
39,Gloucester,2.0,1.0
42,Huddersfield,2.0,1.0


In [218]:

cluster_val = uk_merged.groupby(['Cluster Labels']).sum().reset_index()

cluster_val['Cluster Labels'] = cluster_validation['Cluster Labels'] + 1

uk_merged['Cluster Labels'] = uk_merged['Cluster Labels'] + 1

cluster_validation = cluster_validation.rename(columns={"Asian Restaurant": "Asian_Restaurant_count"})
cluster_validation['Asian_Restaurant_count'] = cluster_validation['Asian_Restaurant_count'].astype(int)
print(cluster_validation[["Asian_Restaurant_count"]])




   Asian_Restaurant_count
0                       0
1                      26
2                      23


Observations:
Cluster 1 : 83 areas with no Asian restaurant
Cluster 2 : 12 areas with 23 Asian restaurant
Cluster 3 : 25 areas with 26 Asian restaurant

We identified cluster 1 and 2 where there are no/very less number of Asian restaurant and hence is suitable for an investment for a new Asian restaurant in the vicinity of those areas.

Cluster 3 areas can be left as they are already having multiple Asian restaurant per area.
